In [1]:
from fastai.vision.all import *

In [2]:
# Download dataset

#path = untar_data(URLs.PETS)

path = untar_data(URLs.CIFAR)

In [3]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()), get_items=get_image_files, get_y=parent_label, item_tfms=Resize(460), batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

In [4]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.319972,1.494396,0.499333,10:45
1,0.864394,0.845482,0.704667,10:39


In [5]:
# Crop and normalize images

def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)

In [6]:
# Image size = 128 (training on smaller images first)

dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.214282,1.418826,0.485083,05:10
1,0.805845,0.755502,0.728167,05:08


In [7]:
# Image size = 224 (training on larger images later)

learn.dls = get_dls(64, 224)
learn.fine_tune(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.087178,1.053990,0.627333,10:40


epoch,train_loss,valid_loss,accuracy,time
0,0.753894,0.762324,0.739583,10:38
1,0.536665,0.503881,0.825500,10:37


##Test time augmentation

In [8]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

0.8341666460037231

In [9]:

model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=MixUp())
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.653646,1.533053,0.448083,05:10
1,1.320174,0.889983,0.689417,05:09
